In [21]:
# an example of random sampling
from Crypto.Cipher import AES
from Crypto.Hash import SHA1
from Crypto.Hash import SHA224
#very stupid error of the library: 
#function/symbol 'SHA_init' not found in library...
#undefined symbol: SHA256_init
#use Cyrptodome instead of Crypto
from Cryptodome.Hash import SHA256
from Crypto.Hash import MD5
from base64 import b64encode
from base64 import b64decode
import time

class SHASH():
    def __init__(self, version):
        self.version = version

    def hash(self, text):
        text = text.encode('utf8')
        # removed SHA-1
        if self.version == '1':
            sha = SHA1.new(data=text)
        elif self.version == '224':
            sha = SHA224.new(data=text)
        elif self.version == '256':
            sha = SHA256.new(data=text)
        elif self.version == 'md5':
            sha = MD5.new(data=text)
        return sha.hexdigest()

def SampleChallengeSet(totalVolume: int, challengeVolume: int, seed: str):
    sampleList = []
    hasher = SHASH('1')
    for i in range(challengeVolume):
        temp = hasher.hash(seed + str(i))
        #print(eval('0x'+temp) % totalVolume)

# challenge rate is 5%, so [(2k,100),(4k,200),...,(10k,500)]        
total = 0
whole_set = 10000
challenge_set = 500
for i in range(0,999):
    t0 = time.time()
    SampleChallengeSet(whole_set, challenge_set, '0xc7881bc9db98067f34655a0635d6facbc770ee0f4b0b18037d0ad3268ddb6b07')
    t1 = time.time()
    total += t1 - t0
print(f"averaged time (total/1k times): {total:0.8f} milliseconds")

averaged time (total/1k times): 6.10821438 milliseconds


In [ ]:
SHA-256 results:
2k/100: 1.51723409 milliseconds
4k/200: 2.96389651 milliseconds
6k/300: 4.55106258 milliseconds
8k/400: 5.95906854 milliseconds
10k/500: 7.42910147 milliseconds

In [16]:
# an example of key enc/dec
from Cryptodome.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Cipher._mode_eax import EaxMode


AESTime = 0
# change the range of i in [2k,4k,6k,8k,10k]
slicing_num = 6000

# change the range of i in [100,200,300,400,500] according to the challenge rate 5% in the settings
sample_num = 300

for i in range(slicing_num): 
    t2 = time.time()
    data = b'hellosdfhdghsdfhdshgsakdhfbasdklfahsdlkfjshadlfsahdifasgdfoahsbdsbklsdfha'
#key_int = 123
#key = key_int.to_bytes(32,'big')
    key = get_random_bytes(32)
#print(key)
    cipher = AES.new(key, AES.MODE_EAX, key)
#print(cipher)

    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(data)
#print(ciphertext)

    t3 = time.time()
    AESTime += t3 - t2
print(f"encryption time: {AESTime:0.8f} seconds")

#cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)

AESDecTime = 0

for i in range(sample_num):
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    t22 = time.time()
    plaintext = cipher.decrypt(ciphertext)
    t33 = time.time()
    AESDecTime += t33 - t22
print(f"decryption time: {AESDecTime:0.8f} seconds")


try:
    cipher.verify(tag)
    print("The message is authentic:", plaintext)
except ValueError:
    print("Key incorrect or message corrupted")

encryption time: 1.36211658 seconds
decryption time: 0.00444460 seconds
The message is authentic: b'hellosdfhdghsdfhdshgsakdhfbasdklfahsdlkfjshadlfsahdifasgdfoahsbdsbklsdfha'


In [142]:
# Python program to illustrate ElGamal encryption
  
import random 
from math import pow
  
a = random.randint(2, 10)
  
def gcd(a, b):
    if a < b:
        return gcd(b, a)
    elif a % b == 0:
        return b;
    else:
        return gcd(b, a % b)
  
# Generating large random numbers
def gen_key(q):
  
    key = random.randint(pow(10, 20), q)
    while gcd(q, key) != 1:
        key = random.randint(pow(10, 20), q)
  
    return key
  
# Modular exponentiation
def power(a, b, c):
    x = 1
    y = a
  
    while b > 0:
        if b % 2 == 0:
            x = (x * y) % c;
        y = (y * y) % c
        b = int(b / 2)
  
    return x % c
  
# Asymmetric encryption
def encrypt(msg, q, h, g):
  
    en_msg = []
  
    k = gen_key(q)# Private key for sender
    s = power(h, k, q)
    p = power(g, k, q)
      
    for i in range(0, len(msg)):
        en_msg.append(msg[i])
    #print("g^k used : ", p)
    #print("g^ak used : ", s)
    
    for i in range(0, len(en_msg)):
        en_msg[i] = s * ord(en_msg[i])
    
  
    return en_msg, p
  
def decrypt(en_msg, p, key, q):
  
    dr_msg = []
    h = power(p, key, q)
    for i in range(0, len(en_msg)):
        dr_msg.append(chr(int(en_msg[i]/h)))
          
    return dr_msg
  
# Driver code
def main():
  
    msg = 'encryption'
    #print("Original Message :", msg)
  
    q = random.randint(pow(10, 20), pow(10, 50))
    g = random.randint(2, q)
  
    key = gen_key(q)# Private key for receiver
    h = power(g, key, q)
    #print("g used : ", g)
    #print("g^a used : ", h)
    
    ElGamalTime = 0
    for i in range(0,999):
        t4 = time.time()

        en_msg, p = encrypt(msg, q, h, g)
    
        t5 = time.time()
        ElGamalTime += t5 - t4
    print(f"encryption time: {ElGamalTime:0.8f} seconds")

    dr_msg = decrypt(en_msg, p, key, q)
    dmsg = ''.join(dr_msg)
    #print("Decrypted Message :", dmsg);
  
  
if __name__ == '__main__':
    main()

encryption time: 0.39425635 seconds
